# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [4]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [5]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

spaceship = pd.get_dummies(spaceship, drop_first=True)

X = spaceship.drop(columns=['Transported'])
y = spaceship['Transported']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(6954, 23735) (6954,)
(1739, 23735) (1739,)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

gb = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid, 
                              scoring='accuracy', cv=5, n_jobs=-1, verbose=2)

grid_search_gb.fit(X_train, y_train)

best_params = grid_search_gb.best_params_
best_model = grid_search_gb.best_estimator_

y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {accuracy_best:.2f}")
print("Best Model Classification Report:\n", classification_report(y_test, y_pred_best))

print("Best Hyperparameters Found:\n", best_params)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


- Evaluate your model

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_best = best_model.predict(X_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {accuracy_best:.2f}")

class_report = classification_report(y_test, y_pred_best)
print("Classification Report:\n", class_report)

conf_matrix = confusion_matrix(y_test, y_pred_best)
print("Confusion Matrix:\n", conf_matrix)

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [ ]:
best_params = {
    'n_estimators': 100,
    'learning_rate': 0.1,
    'max_depth': 4,
    'min_samples_split': 5,
    'min_samples_leaf': 2
}

best_model = GradientBoostingClassifier(random_state=42, 
                                        n_estimators=best_params['n_estimators'],
                                        learning_rate=best_params['learning_rate'],
                                        max_depth=best_params['max_depth'],
                                        min_samples_split=best_params['min_samples_split'],
                                        min_samples_leaf=best_params['min_samples_leaf'])

best_model.fit(X_train, y_train)

y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Fine-Tuned Model Accuracy: {accuracy_best:.2f}")

class_report = classification_report(y_test, y_pred_best)
print("Fine-Tuned Classification Report:\n", class_report)

conf_matrix = confusion_matrix(y_test, y_pred_best)
print("Fine-Tuned Confusion Matrix:\n", conf_matrix)

- Run Grid Search

In [ ]:
gb = GradientBoostingClassifier(random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize Grid Search with cross-validation
grid_search_gb = GridSearchCV(estimator=gb, param_grid=param_grid, 
                              scoring='accuracy', cv=5, n_jobs=-1, verbose=2)

# Fit the Grid Search to the training data
grid_search_gb.fit(X_train, y_train)

# Retrieve the best parameters and estimator
best_params = grid_search_gb.best_params_
best_model = grid_search_gb.best_estimator_

# Evaluate the best model
y_pred_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {accuracy_best:.2f}")

# Detailed evaluation
print("Best Model Classification Report:\n", classification_report(y_test, y_pred_best))

# Output best parameters
print("Best Hyperparameters Found:\n", best_params)

- Evaluate your model

In [ ]:
y_pred_best = best_model.predict(X_test)

accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Best Model Accuracy: {accuracy_best:.2f}")

class_report = classification_report(y_test, y_pred_best)
print("\nClassification Report:\n", class_report)

conf_matrix = confusion_matrix(y_test, y_pred_best)
print("Confusion Matrix:\n", conf_matrix)